In [ ]:
import os
os.chdir("../")

In [ ]:
from datasets import load_dataset
from data import WikiTQHighlightedCellsDataset
import json
from utils import process_config
import torch
import torch.nn as nn
from src import HighlightedCluBartModelForGenerativeQuestionAnswering

In [ ]:
import unicodedata
import numpy as np
import pandas as pd
from tqdm import tqdm
from codecs import open
from math import isnan, isinf
from easydict import EasyDict
from torch.utils.data import DataLoader
from abc import ABCMeta, abstractmethod
import collections
import re

In [ ]:
def normalize(x):

    if not isinstance(x, str):
        x = x.decode('utf8', errors='ignore')

    # Remove diacritics
    x = ''.join(c for c in unicodedata.normalize('NFKD', x)
                if unicodedata.category(c) != 'Mn')
    
    # Normalize quotes and dashes
    x = re.sub(r"[‘’´`]", "'", x)
    x = re.sub(r"[“”]", "\"", x)
    x = re.sub(r"[‐‑‒–—−]", "-", x)

    while True:
        
        old_x = x

        # Remove citations
        x = re.sub(r"((?<!^)\[[^\]]*\]|\[\d+\]|[•♦†‡*#+])*$", "", x.strip())
        
        # Remove details in parenthesis
        x = re.sub(r"(?<!^)( \([^)]*\))*$", "", x.strip())
        
        # Remove outermost quotation mark
        x = re.sub(r'^"([^"]*)"$', r'\1', x.strip())
        
        if x == old_x:
            break
    
    # Remove final '.'
    if x and x[-1] == '.':
        x = x[:-1]
    
    # Collapse whitespaces and convert to lower case
    x = re.sub(r'\s+', ' ', x, flags=re.U).lower().strip()
    
    return x


class Value(object):
    __metaclass__ = ABCMeta

    # Should be populated with the normalized string
    _normalized = None

    @abstractmethod
    def match(self, other):
        """Return True if the value matches the other value.

        Args:
            other (Value)
        Returns:
            a boolean
        """
        pass

    @property
    def normalized(self):
        return self._normalized


class StringValue(Value):

    def __init__(self, content):
        assert isinstance(content, str)
        self._normalized = normalize(content)
        self._hash = hash(self._normalized)

    def __eq__(self, other):
        return isinstance(other, StringValue) and self.normalized == other.normalized

    def __hash__(self):
        return self._hash

    def __str__(self):
        return 'S' + str([self.normalized])

    __repr__ = __str__

    def match(self, other):
        assert isinstance(other, Value)
        return self.normalized == other.normalized


class NumberValue(Value):

    def __init__(self, amount, original_string=None):
        assert isinstance(amount, (int, float))
        if abs(amount - round(amount)) < 1e-6:
            self._amount = int(amount)
        else:
            self._amount = float(amount)
        if not original_string:
            self._normalized = str(self._amount)
        else:
            self._normalized = normalize(original_string)
        self._hash = hash(self._amount)

    @property
    def amount(self):
        return self._amount

    def __eq__(self, other):
        return isinstance(other, NumberValue) and self.amount == other.amount

    def __hash__(self):
        return self._hash

    def __str__(self):
        return ('N(%f)' % self.amount) + str([self.normalized])

    __repr__ = __str__

    def match(self, other):
        assert isinstance(other, Value)
        if self.normalized == other.normalized:
            return True
        if isinstance(other, NumberValue):
            return abs(self.amount - other.amount) < 1e-6
        return False

    @staticmethod
    def parse(text):
        """Try to parse into a number.

        Return:
            the number (int or float) if successful; otherwise None.
        """
        try:
            return int(text)
        except:
            try:
                amount = float(text)
                assert not isnan(amount) and not isinf(amount)
                return amount
            except:
                return None


class DateValue(Value):

    def __init__(self, year, month, day, original_string=None):

        """Create a new DateValue. Placeholders are marked as -1."""
        assert isinstance(year, int)
        assert isinstance(month, int) and (month == -1 or 1 <= month <= 12)
        assert isinstance(day, int) and (day == -1 or 1 <= day <= 31)
        assert not (year == month == day == -1)
        
        self._year = year
        self._month = month
        self._day = day
        
        if not original_string:
            self._normalized = '{}-{}-{}'.format(
                year if year != -1 else 'xx',
                month if month != -1 else 'xx',
                day if day != '-1' else 'xx')
        else:
            self._normalized = normalize(original_string)
        
        self._hash = hash((self._year, self._month, self._day))

    @property
    def ymd(self):
        return (self._year, self._month, self._day)

    def __eq__(self, other):
        return isinstance(other, DateValue) and self.ymd == other.ymd

    def __hash__(self):
        return self._hash

    def __str__(self):
        return (('D(%d,%d,%d)' % (self._year, self._month, self._day))
                + str([self._normalized]))

    __repr__ = __str__

    def match(self, other):
        
        assert isinstance(other, Value)
        
        if self.normalized == other.normalized:
            return True
        
        if isinstance(other, DateValue):
            return self.ymd == other.ymd
        
        return False

    @staticmethod
    def parse(text):
        """Try to parse into a date.

        Return:
            tuple (year, month, date) if successful; otherwise None.
        """
        try:
            ymd = text.lower().split('-')
            assert len(ymd) == 3
            year = -1 if ymd[0] in ('xx', 'xxxx') else int(ymd[0])
            month = -1 if ymd[1] == 'xx' else int(ymd[1])
            day = -1 if ymd[2] == 'xx' else int(ymd[2])
            assert not (year == month == day == -1)
            assert month == -1 or 1 <= month <= 12
            assert day == -1 or 1 <= day <= 31
            return (year, month, day)
        except:
            return None


def to_value(original_string, corenlp_value=None):
    """Convert the string to Value object.

    Args:
        original_string (basestring): Original string
        corenlp_value (basestring): Optional value returned from CoreNLP
    Returns:
        Value
    """

    if isinstance(original_string, Value):
        # Already a Value
        return original_string
    
    if not corenlp_value:
        corenlp_value = original_string
    
    # Number?
    amount = NumberValue.parse(corenlp_value)
    
    if amount is not None:
        return NumberValue(amount, original_string)
    
    # Date?
    ymd = DateValue.parse(corenlp_value)
    if ymd is not None:
        if ymd[1] == ymd[2] == -1:
            return NumberValue(ymd[0], original_string)
        else:
            return DateValue(ymd[0], ymd[1], ymd[2], original_string)
    
    # String.
    return StringValue(original_string)


def to_value_list(original_strings, corenlp_values=None):
    """Convert a list of strings to a list of Values

    Args:
        original_strings (list[basestring])
        corenlp_values (list[basestring or None])
    Returns:
        list[Value]
    """
    assert isinstance(original_strings, (list, tuple, set))
    if corenlp_values is not None:
        assert isinstance(corenlp_values, (list, tuple, set))
        assert len(original_strings) == len(corenlp_values)
        return list(set(to_value(x, y) for (x, y)
                        in zip(original_strings, corenlp_values)))
    else:
        return list(set(to_value(x) for x in original_strings))


def check_denotation(target_values, predicted_values):
    """Return True if the predicted denotation is correct.

    Args:
        target_values (list[Value])
        predicted_values (list[Value])
    Returns:
        bool
    """

    # Check size
    if len(target_values) != len(predicted_values):
        return False
    
    # Check items
    for target in target_values:
        if not any(target.match(pred) for pred in predicted_values):
            return False
    
    return True


def tsv_unescape(x):
    """Unescape strings in the TSV file.
    Escaped characters include:
        newline (0x10) -> backslash + n
        vertical bar (0x7C) -> backslash + p
        backslash (0x5C) -> backslash + backslash

    Args:
        x (str or unicode)
    Returns:
        a unicode
    """
    return x.replace(r'\n', '\n').replace(r'\p', '|').replace('\\\\', '\\')


def tsv_unescape_list(x):
    """Unescape a list in the TSV file.
    List items are joined with vertical bars (0x5C)

    Args:
        x (str or unicode)
    Returns:
        a list of unicodes
    """
    return [tsv_unescape(y) for y in x.split('|')]

In [ ]:
with open('./dater_outputs/gloc_wtq_end2end_wikitq_test.json','r') as f:
    dic = json.load(f)

In [ ]:
dater_error_indices = []

In [ ]:
for key in dic:

    id = key
    to_union = collections.defaultdict(float)
    it = dic[key]
    table = it['data_item']['table_text']
    st = it['data_item']['statement']
    ######### col filed################
    preds = it['generations']
    actual_answer = it['data_item']['answer']
    
    for pred in preds:
        log_prob_mean = pred[2]
        pred = pred[0]

        pred = pred.split('therefore,the answer is :')[-1]
        
        key = pred
        to_union[key] += np.exp(log_prob_mean)
    d_ordered = sorted(to_union.items(),key=lambda x:x[1],reverse=True)
    try:
        pred_answer = d_ordered[0][0].split('\n')[0].strip()
    except Exception:
        pred_answer = 'error'
    st = st.split('\n')[0]
    # treat pred as an single entity because of 2 prompt with out example
    # if len(target_values) == 1:
    #     pred_answer = [pred_answer]
    # else:
    #     pred_answer = pred_answer.split(',')
    
    # treat pred answer as only on entity
    pred_answer = [pred_answer]
    pred_answer = to_value_list(pred_answer)

    actual_answer = to_value_list(actual_answer)

    correct = check_denotation(actual_answer, pred_answer)

    if not correct:
        dater_error_indices.append(id)

In [ ]:
with open("configs/wiki_tq/tapex.json", "rb") as f:
    config = json.load(f)

config = process_config(config)

In [ ]:
dataset = load_dataset(config.data.data_path)

In [ ]:
test_dataset = WikiTQHighlightedCellsDataset(dataset=dataset, config=config, data_type="test")

In [ ]:
tokenizer = test_dataset.tokenizer

In [ ]:
model = HighlightedCluBartModelForGenerativeQuestionAnswering(config=config)
model.load_state_dict(torch.load("omnitab_best_ckpt/epoch=28.pt", map_location="cpu"))

In [ ]:
model.to("cuda:0")

In [ ]:
def predict(index):

    input_ids, attention_mask, token_type_ids, decoder_input_ids, highlighted_cells, labels = test_dataset.__getitem__(index)
    input_ids = input_ids.unsqueeze(0).to("cuda:0")
    attention_mask = attention_mask.unsqueeze(0).to("cuda:0")
    decoder_input_ids = decoder_input_ids.unsqueeze(0).to("cuda:0")
    highlighted_cells = highlighted_cells.unsqueeze(0).to("cuda:0")
    labels = labels.unsqueeze(0).to("cuda:0")

    inputs_embeds = model.model.model.decomposer.embed_tokens(input_ids) * model.model.model.decomposer.embed_scale

    decomposer_outputs = model.model.model.decomposer(input_ids=None,
            attention_mask=attention_mask,
            head_mask=None,
            inputs_embeds=inputs_embeds,
            output_attentions=None,
            output_hidden_states=None,
            return_dict=None,
        )
    
    latent_rep = model.model.model.latent_rep_head(decomposer_outputs[0])
    # cluster_labels = torch.norm(latent_rep.unsqueeze(2) - model.model.model.cluster_centers.unsqueeze(0).unsqueeze(0), dim = -1).squeeze().argmin(dim = -1)

    soft_labels_numerator = (1 + torch.norm((latent_rep.unsqueeze(2) - model.model.model.cluster_centers.unsqueeze(0).unsqueeze(0)), dim = -1) / model.model.model.clu_alpha) ** (-(1 + model.model.model.clu_alpha) / 2)
    soft_labels = soft_labels_numerator / torch.sum(soft_labels_numerator, dim = -1).unsqueeze(-1)

    token_scores_1 = model.model.model.token_classifier_score1(latent_rep)
    token_scores_2 = model.model.model.token_classifier_score2(latent_rep)
    gaussian_rvs = model.model.gaussian_dist.sample(token_scores_1.shape).to(token_scores_1.device)
    relevance_logit = gaussian_rvs * token_scores_1 + token_scores_2
    relevance_score = model.model.model.sigmoid(relevance_logit)

    # NOTE: Uncomment as per requirement of the experiment
    
    relevance_score = (0.7 * relevance_score + 0.3 * highlighted_cells.unsqueeze(-1)).squeeze()

    print(test_dataset.tokenizer.decode(input_ids[0], skip_special_tokens = True))
    print(test_dataset.tokenizer.decode(input_ids[0][relevance_score >= relevance_score.mean()]))
    print(test_dataset.tokenizer.decode(labels[labels != -100]))

    output_ids = model.model.generate(input_ids = input_ids.to("cuda:0"), max_new_tokens = config.tokenizer.output_max_length, 
                            num_beams = 3, early_stopping = True, attention_mask = attention_mask.to("cuda:0"), 
                            highlighted_cells = highlighted_cells.to("cuda:0")).squeeze()
    
    print(tokenizer.decode(output_ids, skip_special_tokens=True))

In [ ]:
for i, id in enumerate(dater_error_indices):


    if i < 47:
        continue

    print(id)
    predict(int(id))
    
    to_union = collections.defaultdict(float)
    it = dic[id]
    table = it['data_item']['table_text']
    st = it['data_item']['statement']
    ######### col filed################
    preds = it['generations']
    actual_answer = it['data_item']['answer']
    
    for pred in preds:
        log_prob_mean = pred[2]
        pred = pred[0]

        pred = pred.split('therefore,the answer is :')[-1]
        
        key = pred
        to_union[key] += np.exp(log_prob_mean)
    d_ordered = sorted(to_union.items(),key=lambda x:x[1],reverse=True)
    try:
        pred_answer = d_ordered[0][0].split('\n')[0].strip()
    except Exception:
        pred_answer = 'error'

    print("DATER prediction: ", pred_answer)
    print("DATER statement: ", st)

    print("DATER table", table)
    table_column_names = table[0]
    table_content_values = table[1:]
    
    table_df = pd.DataFrame.from_dict({str(col).lower(): [str(table_content_values[j][i]).lower() for j in range(len(table_content_values))] for i, col in enumerate(table_column_names)})

    display(table_df)


    print("Actual table")

    table_column_names = dataset["test"][int(id)]["table"]["header"]
    table_content_values = dataset["test"][int(id)]["table"]["rows"]
    actual_table_df = pd.DataFrame.from_dict({str(col).lower(): [str(table_content_values[j][i]).lower() for j in range(len(table_content_values))] for i, col in enumerate(table_column_names)})

    display(actual_table_df)
    break